<a href="https://colab.research.google.com/github/ramdatascience/classificiation-model-using-Keras-Tuner/blob/master/keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.3.0


In [4]:
f_mnist=keras.datasets.fashion_mnist

In [5]:
# Loading the Datasets for Data and Lables.
(train_images,train_labels),(test_images,test_labels)=f_mnist.load_data()

In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
# Shape of the Train Dataset.
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [9]:
# model building with dynamic Filter and neuron values with hyperparameter tuning using the Keras tuner(hp.int, hp.choice).
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json
INFO:tensorflow:Reloading Tuner from output/Mnist Fashion/tuner0.json


In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

INFO:tensorflow:Oracle triggered exit


In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 48)        1248      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 32)        38432     
_________________________________________________________________
flatten (Flatten)            (None, 12800)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                819264    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 859,594
Trainable params: 859,594
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.1871 - accuracy: 0.9294 - val_loss: 0.2428 - val_accuracy: 0.9158
Epoch 5/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.1539 - accuracy: 0.9418 - val_loss: 0.2691 - val_accuracy: 0.9103
Epoch 6/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.1239 - accuracy: 0.9536 - val_loss: 0.2825 - val_accuracy: 0.9122
Epoch 7/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.1012 - accuracy: 0.9621 - val_loss: 0.2724 - val_accuracy: 0.9182
Epoch 8/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0844 - accuracy: 0.9682 - val_loss: 0.3152 - val_accuracy: 0.9120
Epoch 9/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0676 - accuracy: 0.9746 - val_loss: 0.3705 - val_accuracy: 0.9137
Epoch 10/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0598 - accuracy: 0.9777 - val_loss: 0.4584 - val_a